# ShallowNeuralNetwork(PyTorch)

In [1]:
import torch 
from sklearn import datasets
from torch.autograd import Variable
from sklearn.model_selection import train_test_split

### 1 Load Data

这里我们使用sklearn中的iris数据集.

In [2]:
def loadData_iris():
    """
    Returns:
    --------
        X:have two dimensions (sepal length and width).
        Y:labels.
    """
    iris = datasets.load_iris()
    X = iris.data[:100, :2]
    Y = iris.target[:100]
    return X,Y

In [3]:
X,y = loadData_iris()
print('X shape is:',X.shape)
print('y shape is:',y.shape)

X shape is: (100, 2)
y shape is: (100,)


#### 1.1 Split data set

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

print('X_train shape:',X_train.shape)
print('X_test shape:',X_test.shape)
print('y_train shape:',y_train.shape)
print('y_test shape:',y_test.shape)

X_train shape: (80, 2)
X_test shape: (20, 2)
y_train shape: (80, 1)
y_test shape: (20, 1)


### 2 Build Pytorch Model

#### 2.1 change type to tensor

In [6]:
X_train = Variable(torch.Tensor(X_train))
X_test = Variable(torch.Tensor(X_test))
y_train = Variable(torch.Tensor(y_train))
y_test = Variable(torch.Tensor(y_test))

在使用线性计算时需要注意:

```python
torch.nn.Linear(in_features, out_features, bias=True)```

**Example:**

```python
m = nn.Linear(20, 30)
input = torch.randn(128, 20)
output = m(input)
print(output.size())
torch.Size([128, 30])```


可以看到,如果```in_features=20,out_features=30```,那么最后的输出为30.也就是说,最后输出的是shape(m,out_features).

那么我们喂入的样本形状需要是(m,n)的形式.

以我们现在的案例来讲:

```self.line1 = torch.nn.Linear(n,4)```输出的是shape(m,4)

```self.line2 = torch.nn.Linear(4,1)```输出的是shape(m,1)

所以我们的labels形状需要是(m,1).这样我们才可以进一步计算loss.


#### 2.2 build class

(1) 定义hidden layer-1:

```python

self.line1 = torch.nn.Linear(n,4)
Z1 = self.line1(X)
A1 = torch.relu(Z1)```

(2) 定义hidden layer-2:

```python
self.line2 = torch.nn.Linear(4,1)
Z2 = self.line2(A1)
A2 = torch.sigmoid(Z2)
```

In [7]:
class shallowNN(torch.nn.Module):
    """
    torch class:define NN
    Parameters:
    ----------
        n: number features.
    Return:
    ------
        A2: layer-2:sigmoid value.
    """
    def __init__(self,n):
        torch.nn.Module.__init__(self)
        self.line1 = torch.nn.Linear(n,4) # linear function,Note this parameters.
        self.line2 = torch.nn.Linear(4,1)
        
    def forward(self,X):
        """
        Forward propagation
        Prameters:
        ---------
            X: data set
        """
        Z1 = self.line1(X)
        A1 = torch.relu(Z1)
        Z2 = self.line2(A1)
        A2 = torch.sigmoid(Z2)
        return A2

#### 2.3 Build SNN

(1) 初始化模型:

```python
model = shallowNN(n)```

(2) 初始化二分类损失熵:

```python
ceriterion = torch.nn.BCELoss(reduction='mean')```

(3) 初始化优化器:

```python
optimizer = torch.optim.SGD(model.parameters(),lr=alpha)```

(4) 重置梯度值:

Pytorch在梯度计算方面是使用累加的方式

```python
optimizer.zero_grad()```

(5) 计算反向传播:

```python
loss.backward()```

(6) 更新所有参数:

```python
optimizer.step()```

(7) 返回Forward以便于测试正确率:

```python
model.forward```

In [8]:
def ShallowNN_model(X,y,alpha,Iter,is_print=False):
    """
    Build SNN
    
    parameters:
    ----------
       X: training data set
       y: training labels
       alpha: learning rate
       Iter: iterative number
       is_print: print loss.
       
    Return:
    ------
       model.forward: forward propagation: caclulate accuracy. 
    """
    m,n = X.shape
    model = shallowNN(n)
    
    ceriterion = torch.nn.BCELoss(reduction='mean')
    optimizer = torch.optim.SGD(model.parameters(),lr=alpha)
    
    for iter_ in range(Iter):
        
        A2 = model.forward(X)
        loss = ceriterion(A2,y)
        
        if iter_ % 100 ==0 and is_print:
            print('After iter {} loss: {}'.format(iter_,loss))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    
    return model.forward
    

In [9]:
forward = ShallowNN_model(X_train,y_train,0.1,300,True)

After iter 0 loss: 0.7204402089118958
After iter 100 loss: 0.3151532709598541
After iter 200 loss: 0.1036759465932846


#### 2.4 predict and score

下面查看一下测试,训练样本的正确率.

In [10]:
def predict_score(X,y,forward_func):
    """
    predict and score
    Parameters:
    ----------
        X: data set
        y: labels
        forward_func: forward propagation
    
    Return:
    ------
        accuracy: correct rate.
    """
    m,n = X.shape
    predict_y = torch.round(forward_func(X))
    accuracy = torch.sum((predict_y == y)).item() / m
    
    return accuracy

In [11]:
accuracy = predict_score(X_test,y_test,forward)
print('The test data set accuracy:',accuracy)

The test data set accuracy: 1.0


In [12]:
accuracy = predict_score(X_train,y_train,forward)
print('The train data set accuracy:',accuracy)

The train data set accuracy: 0.9875
